Part 1: Data Ingestion

In [2]:
#Downloading the data

import requests
import os 

#This is the directory where the files are stored
os.makedirs("data/raw", exist_ok=True)

#We then make a function to call to download each file
def download_file(url, destination_path):
    print(f"Beginning download of {url}...")
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status() # Check if request was successful

        #We then open the file to download the data to
        with open(destination_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)

        print(f"Download completed and saved to {destination_path}.")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while downloading {url}: {e}")

In [3]:
#Using our previous download helper function, we download the two required files

yellow_taxi_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet"
taxi_zone_url = "https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv"

download_file(yellow_taxi_url, "data/raw/yellow_tripdata_2024-01.parquet")
download_file(taxi_zone_url, "data/raw/taxi_zone_lookup.csv")

print("All files downloaded successfully.")

Beginning download of https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet...
Download completed and saved to data/raw/yellow_tripdata_2024-01.parquet.
Beginning download of https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv...
Download completed and saved to data/raw/taxi_zone_lookup.csv.
All files downloaded successfully.


In [4]:
#We now load the data in a polars dataframe for data validation
#Polars is used for further processing due to its speed advantages over pandas for larger datasets such as this one

import polars as pl

#Load the datasets
taxi_trip_df = pl.read_parquet("data/raw/yellow_tripdata_2024-01.parquet")
taxi_zone_df = pl.read_csv("data/raw/taxi_zone_lookup.csv")

In [5]:
#Verifying all expected columns exist

expected_columns = {
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime",
    "PULocationID",
    "DOLocationID",
    "passenger_count",
    "trip_distance",
    "fare_amount",
    "tip_amount",
    "total_amount",
    "payment_type",
}

for col in expected_columns:
    if col not in taxi_trip_df.columns:
        raise Exception(f"Missing column: {col} in the taxi trip dataset, aborting...")


In [6]:
#Checking date columns are of valid datetime type

date_columns = ["tpep_pickup_datetime", "tpep_dropoff_datetime"]

for col in date_columns:
    #The below if state checks if the column for the two essential date columns is of datetime type of the various measurement units
    if taxi_trip_df[col].dtype not in (pl.Datetime, pl.Datetime("us"), pl.Datetime("ms"), pl.Datetime("ns")):
        raise Exception(f"Column {col} is not of datetime type, aborting...")
    

In [7]:
#Printing a summary of the data validation to the console
print("Summary of Data Validation: \n")

print("All expected columns are present in the taxi trip dataset.")
print("Date columns are of valid datetime type in taxi trip dataset. \n")

print(f"Total number of rows in taxi trip dataset: {len(taxi_trip_df)}")
print(f"Total number of rows in taxi zone dataset: {len(taxi_zone_df)}\n")

print(f"Taxi trip dataset schema: \n {taxi_trip_df.schema}")
print(f"Taxi zone dataset schema: \n {taxi_zone_df.schema}\n")

print("Data validation completed successfully.")

Summary of Data Validation: 

All expected columns are present in the taxi trip dataset.
Date columns are of valid datetime type in taxi trip dataset. 

Total number of rows in taxi trip dataset: 2964624
Total number of rows in taxi zone dataset: 265

Taxi trip dataset schema: 
 Schema([('VendorID', Int32), ('tpep_pickup_datetime', Datetime(time_unit='ns', time_zone=None)), ('tpep_dropoff_datetime', Datetime(time_unit='ns', time_zone=None)), ('passenger_count', Int64), ('trip_distance', Float64), ('RatecodeID', Int64), ('store_and_fwd_flag', String), ('PULocationID', Int32), ('DOLocationID', Int32), ('payment_type', Int64), ('fare_amount', Float64), ('extra', Float64), ('mta_tax', Float64), ('tip_amount', Float64), ('tolls_amount', Float64), ('improvement_surcharge', Float64), ('total_amount', Float64), ('congestion_surcharge', Float64), ('Airport_fee', Float64)])
Taxi zone dataset schema: 
 Schema([('LocationID', Int64), ('Borough', String), ('Zone', String), ('service_zone', String)]

Part 2: Data Transformation & Analysis

In [8]:
#We will now sanitize the data for preparation for analysis

#First we clean up any rows with null values in important columns such as pick and dropoff times, locations, fares and trips

important_columns = [
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime",
    "PULocationID",
    "DOLocationID",
    "fare_amount",
    "trip_distance"
]

row_count_before_null_filter = len(taxi_trip_df)

taxi_trip_df = taxi_trip_df.drop_nulls(subset=important_columns)

#Record the number of rows removed for later documentation
removed_null_rows = row_count_before_null_filter - len(taxi_trip_df)

In [9]:
#We now clean the data by removing any rows where the trip has zero or negative distance, negative fares, or fares exceeding $500

rows_before_invalid_filter = len(taxi_trip_df)

taxi_trip_df = taxi_trip_df.filter(
    pl.col("trip_distance") > 0,
    pl.col("fare_amount") >= 0,
    pl.col("fare_amount") <= 500
)

removed_invalid_rows = rows_before_invalid_filter - len(taxi_trip_df)

In [10]:
#For our last bit of sanitization we then remove rows where dropoff time is before pickup time

rows_before_dropoff_filter = len(taxi_trip_df)

taxi_trip_df = taxi_trip_df.filter(
    pl.col("tpep_dropoff_datetime") >= pl.col("tpep_pickup_datetime")
)

removed_dropoff_rows = rows_before_dropoff_filter - len(taxi_trip_df)

In [11]:
#Documenting the sanitization results

print("Data Sanitization Summary: \n")

print(f"Total rows before sanitization were: {row_count_before_null_filter} \n")

print(f"Rows removed due to null values in important columns: {removed_null_rows}")
print(f"Rows remaining after removing null values were: {rows_before_invalid_filter} \n")

print(f"Rows removed due to invalid trip distance or fare amount: {removed_invalid_rows}")
print(f"Rows remaining after removing invalid trip distance or fare amount were: {rows_before_dropoff_filter}\n")

print(f"Rows removed due to dropoff time before pickup time: {removed_dropoff_rows}\n")

print(f"Total rows removed during sanitization were: {removed_null_rows + removed_invalid_rows + removed_dropoff_rows}")
print(f"Total remaining rows in dataset: {len(taxi_trip_df)} \n")

Data Sanitization Summary: 

Total rows before sanitization were: 2964624 

Rows removed due to null values in important columns: 0
Rows remaining after removing null values were: 2964624 

Rows removed due to invalid trip distance or fare amount: 94466
Rows remaining after removing invalid trip distance or fare amount were: 2870158

Rows removed due to dropoff time before pickup time: 56

Total rows removed during sanitization were: 94522
Total remaining rows in dataset: 2870102 



In [12]:
#Next we do feature engineering to add our own derived columns to the dataset

#Adding trip duration in minutes
taxi_trip_df = taxi_trip_df.with_columns([
    ((pl.col("tpep_dropoff_datetime") - pl.col("tpep_pickup_datetime"))
     .dt.total_seconds() / 60)
     .alias("trip_duration_minutes")
])

#Adding trip speed in mph
taxi_trip_df = taxi_trip_df.with_columns([
    pl.when(pl.col("trip_duration_minutes") > 0)
      .then(pl.col("trip_distance") / (pl.col("trip_duration_minutes") / 60))
      .otherwise(None)
      .alias("trip_speed_mph")
])

#Adding pickup hour
taxi_trip_df = taxi_trip_df.with_columns([
    pl.col('tpep_pickup_datetime').dt.hour().alias('pickup_hour')
])

#Adding pickup day of week
taxi_trip_df = taxi_trip_df.with_columns([
    pl.col('tpep_pickup_datetime').dt.strftime("%A").alias('pickup_day_of_week')
])


What are the top 10 busiest pickup zones by total number of trips?

In [26]:
import duckdb

# Create a DuckDB connection
con = duckdb.connect()

#Register dataframes
con.register("taxi_trips", taxi_trip_df)
con.register("taxi_zones", taxi_zone_df)

busiest_pickup_zones = con.execute('''
    SELECT 
        z."Zone" AS pickup_zone,
        COUNT(*) AS total_trips
    FROM
        taxi_trips t
    JOIN
        taxi_zones z 
            ON t.PULocationID = z.LocationID
    GROUP BY 
        z."Zone"
    ORDER BY 
        total_trips DESC
    LIMIT 10
                                  
'''
).fetchdf()

print(busiest_pickup_zones)

                    pickup_zone  total_trips
0                Midtown Center       140161
1         Upper East Side South       140134
2                   JFK Airport       138478
3         Upper East Side North       133975
4                  Midtown East       104356
5     Times Sq/Theatre District       102972
6  Penn Station/Madison Sq West       102161
7           Lincoln Square East       101800
8             LaGuardia Airport        87715
9         Upper West Side South        86475


 What is the average fare amount for each hour of the day?

In [19]:
average_fair = con.execute(''' 
    SELECT 
        pickup_hour,
        AVG(fare_amount) as avg_fare_amount
    FROM
        taxi_trips
    GROUP BY 
       pickup_hour
    ORDER BY 
        pickup_hour
''').fetchdf()

print(average_fair)

    pickup_hour  avg_fare_amount
0             0        19.679250
1             1        17.732032
2             2        16.621723
3             3        18.530033
4             4        23.435229
5             5        27.492713
6             6        22.026585
7             7        18.749879
8             8        17.822939
9             9        17.943989
10           10        18.047523
11           11        17.628112
12           12        17.796520
13           13        18.418805
14           14        19.271523
15           15        19.110366
16           16        19.457290
17           17        18.118545
18           18        17.013712
19           19        17.626564
20           20        18.050403
21           21        18.292862
22           22        19.110051
23           23        20.243498


What percentage of trips use each payment type?


In [ ]:
payment_type_perc = con.execute(''' 
    SELECT 
        payment_type,        
        CASE payment_type
            WHEN 1 THEN 'Credit card'
            WHEN 2 THEN 'Cash'
            WHEN 3 THEN 'No charge'
            WHEN 4 THEN 'Dispute'
            WHEN 5 THEN 'Unknown'
            ELSE 'Other'
        END AS payment_method,
        COUNT(*) * 100.0 / SUM(COUNT(*)) OVER () AS percentage_of_trips
    FROM
        taxi_trips
    GROUP BY 
        payment_type
    ORDER BY
        payment_type                    
''').fetchdf()

print(payment_type_perc)

   payment_type payment_method  percentage_of_trips
0             0          Other             4.015363
1             1    Credit card            80.081196
2             2           Cash            14.735400
3             3      No charge             0.370997
4             4        Dispute             0.797045


What is the average tip percentage (tip_amount/fare_amount) by day of week, for
credit card payments only?

In [ ]:
avg_tip_perc = con.execute(''' 
    SELECT 
        pickup_day_of_week,
        AVG(tip_amount/NULLIF(fare_amount, 0)) * 100 as avg_tip_percentage
    FROM
        taxi_trips
    WHERE
        payment_type = 1
    GROUP BY 
        pickup_day_of_week
    ORDER BY
        pickup_day_of_week                    
''').fetchdf()

print(avg_tip_perc)

  pickup_day_of_week  avg_tip_percentage
0             Friday           25.595719
1             Monday           25.513977
2           Saturday           26.293897
3             Sunday           25.100984
4           Thursday           29.734458
5            Tuesday           25.729989
6          Wednesday           25.706582


What are the top 5 most common pickup-dropoff zone pairs?

In [28]:
pickup_dropoff_pair = con.execute(''' 
    SELECT 
        pick_up."Zone" as pickup_zone,
        drop_off."Zone" as dropoff_zone,
        COUNT(*) AS total_trips
    FROM
        taxi_trips t
    JOIN taxi_zones pick_up
        ON t.PULocationID = pick_up.LocationID
    JOIN taxi_zones drop_off
        ON t.DOLocationID = drop_off.LocationID
    GROUP BY 
        pickup_zone, 
        dropoff_zone
    ORDER BY 
        total_trips DESC    
    LIMIT 5
''').fetchdf()

print(pickup_dropoff_pair)

             pickup_zone           dropoff_zone  total_trips
0  Upper East Side South  Upper East Side North        21642
1  Upper East Side North  Upper East Side South        19199
2  Upper East Side North  Upper East Side North        15200
3  Upper East Side South  Upper East Side South        14119
4         Midtown Center  Upper East Side South        10139
